# Creación de examen de Seguridad Informática

In [ ]:
[ $(whoami) == "pi" ] # Ejecutamos como Pi
echo $?               # Salida debería ser "0"

sudo apt install steghide wipe hostapd isc-dhcp-server haveged apache2 git acl
sudo systemctl enable isc-dhcp-server.service
sudo systemctl enable haveged
sudo systemctl enable apache2

git clone https://github.com/TryCatchHCF/Cloakify.git

cat << EOF | sudo tee /etc/hostapd/hostapd.conf
ssid=gcs-examen-seguridad
wpa_passphrase=1n2i8k1a02121992
interface=wlan0
auth_algs=3
channel=7
driver=nl80211
hw_mode=g
logger_stdout=-1
logger_stdout_level=2
max_num_sta=5
rsn_pairwise=CCMP
wpa=2
wpa_key_mgmt=WPA-PSK
wpa_pairwise=TKIP CCMP
ieee80211n=1
ieee80211d=1
wmm_enabled=0
country_code=US
EOF

cat << EOF | sudo tee /etc/dhcp/dhcpd.conf
option domain-name "examen.org";
default-lease-time 600;
max-lease-time 7200;
ddns-update-style none;
authoritative;
subnet 10.42.66.0 netmask 255.255.255.0 {
  range 10.42.66.10 10.42.66.80;
}
EOF

sudo sed -i 's/INTERFACESv4=""/INTERFACESv4="wlan0"/' /etc/default/isc-dhcp-server

cat << EOF | sudo tee /etc/network/interfaces.d/wlan0
auto wlan0
iface wlan0 inet static
  address 10.42.66.118
  netmask 255.255.255.0
EOF


sudo apt install libpam-google-authenticator
sudo useradd -m greencore
# 1n2i8k1a02121992
sudo usermod -p '$6$Jo48SOJH$SEWh212t5jWvPksGen23srD1SuSyFeweB.LHulzv8SI1MPHJ0vrmbK6pycBtb5aKtm5KwHEiU8rUNznMZmoLy0' greencore
cat << EOF | sudo tee ~greencore/.google_authenticator
N7FBABAILNOJ2C3U7DC6WDU2UE
" RATE_LIMIT 3 30
" WINDOW_SIZE 17
" DISALLOW_REUSE
" TOTP_AUTH
77278718
13560859
78691076
57681135
13628439
EOF
sudo chown -v greencore:greencore ~greencore/.google_authenticator
sudo chmod 0600 ~greencore/.google_authenticator

cat << EOF | sudo tee -a /etc/pam.d/sshd
auth required pam_google_authenticator.so nullok
EOF
sudo sed -i 's/^ChallengeResponseAuthentication no/ChallengeResponseAuthentication yes/' /etc/ssh/sshd_config
sudo systemctl restart ssh.service

sudo apt install python-pip
sudo pip2 install xortool
wget -O /tmp/pg48871.txt https://www.gutenberg.org/cache/epub/48871/pg48871.txt
xortool-xor -f /tmp/pg48871.txt -s "1n2i8k1a02121992" | sudo tee /home/greencore/.readme > /dev/null
sudo rm /var/www/html/index.html


sudo systemctl unmask hostapd
sudo systemctl enable hostapd

# Debe empotrar el qr-code en alguna imagen, y moverlas a /var/www/html/
# Reinicie la Pi
# TODO: el AP levanta manualmente